##   Grammarly <br> <font size='4'>Monthly Users Analysis</font> <br> <font size='3'>Febraury 2016<br> <font size='2'> Reza Taeb - Data Scientist Candidate - Grammarly </font>

In [1]:
# import essential libraries and packages

import pandas as pd
import numpy as np
import re
import timeit 
import seaborn as sns
import datetime
from datetime import datetime
import matplotlib.pyplot as plt
import altair as alt
import warnings
warnings.filterwarnings('ignore')

In [2]:
# read JSON file 

df = pd.read_json("grammarly_data_exercise.json")

In [3]:
# dataframe summary

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4823567 entries, 0 to 4823566
Data columns (total 5 columns):
 #   Column     Dtype         
---  ------     -----         
 0   date       datetime64[ns]
 1   timestamp  datetime64[ns]
 2   uid        object        
 3   isFirst    bool          
 4   utmSource  object        
dtypes: bool(1), datetime64[ns](2), object(2)
memory usage: 151.8+ MB


In [89]:
df.head(5)

,date,timestamp,uid,isFirst,utmSource
0,2016-02-01,2016-02-01 08:00:01.230,53386875,False,Biznesowe+Rewolucje
1,2016-02-01,2016-02-01 08:00:01.410,70935792,False,None
2,2016-02-01,2016-02-01 08:00:01.451,IG4TX0lZdTk1,False,None
3,2016-02-01,2016-02-01 08:00:01.556,70128721,False,Biznesowe+Rewolucje
4,2016-02-01,2016-02-01 08:00:01.673,53386875,False,Biznesowe+Rewolucje


The '.head' method shows a limited number of rows; if we want to get a better sense of the dataset, we need to see more rows. 

In [5]:
# Function to check more rows of df
def display_all(data):
    
    '''This function shows more rows'''
    
    with pd.option_context("display.min_rows", 20): 
        display(data)

In [6]:
display_all(df)

,date,timestamp,uid,isFirst,utmSource
0,2016-02-29,2016-02-29 08:00:01.255,g4Hjpoji4dmv,False,Biznesowe+Rewolucje
1,2016-02-29,2016-02-29 08:00:01.556,g4Hjpoji4dmv,False,Biznesowe+Rewolucje
2,2016-02-29,2016-02-29 08:00:01.770,75121338,False,program
3,2016-02-29,2016-02-29 08:00:03.129,11l0tD2wFPFO,False,None
4,2016-02-29,2016-02-29 08:00:03.610,56429432,False,None
5,2016-02-29,2016-02-29 08:00:03.620,53677597,False,Biznesowe+Rewolucje
6,2016-02-29,2016-02-29 08:00:03.694,53171302,False,program
7,2016-02-29,2016-02-29 08:00:03.834,60022150,False,answers
8,2016-02-29,2016-02-29 08:00:03.869,53677597,False,Biznesowe+Rewolucje
9,2016-02-29,2016-02-29 08:00:03.930,66538731,False,program


In [7]:
# Count total NaN of each column in a DataFrame

print("Total counts of NaN at each column in a DataFrame:\n\n",df.isnull().sum(),sep = '')

Total counts of NaN at each column in a DataFrame:

date               0
timestamp          0
uid                0
isFirst            0
utmSource    1674386
dtype: int64


The only attribute that has NaN entries is **UtmSource**. I will take care of it later.

The first step should be sorting the dataframe based on the ***timestamp***. Since the dataset is relatively big, it would be better to use **Merge Sort** since it is slightly faster than 'Qucik Sort*.

In [8]:
%%time

df.sort_values(by=['timestamp'], inplace=True, kind='mergesort')
df = df.reset_index(drop= True)

CPU times: user 737 ms, sys: 180 ms, total: 916 ms
Wall time: 922 ms


## Pre Processing Steps

<font size='4'>
Before jumping into questions, we need to do some preprocessing steps:
</font>

<br>
<br>
<font color='darkblue'>

- Removing Exact Duplications 
- Removing In Practice Duplications (for instance: same [date/timestamp/uid] )
- Checking the Entries With More Than One 'isFirst' 
- Checking the Entires With 'isFirst' not at their first appearance (delayed isFirst record)

</font>
<br>
** The other data wrangling part (modifiying utmSource) will be done later for question three.

#### A -  Remove Duplications

Keeping duplications is one of the most common mistakes that analysts make. 

In [9]:
# Check the number of duplications
print ("The number of exact duplications in original dataset:",df.duplicated().sum())

# remove the duplicates 
df_clean = df.loc[~df.duplicated(),:]

The number of exact duplications in original dataset: 12979


#### B - Remove In Practice Duplications 

The dataset had **12979** exact duplications.<br>
However, we need to dig more into the data since it may still have some duplications, which are not exact duplications, but in practice, they are duplications or occurred because of some malfunctions.

For instance, two entries may have exact ***"date / timestamp / uid"***, which can not be true in practice since two pings from the same user can not occur in 1 millisecond. 

In [10]:
# Check duplication of [date/timestamp/uid] combination
print("Number of same [date/timestamp/uid] combination:", df_clean.duplicated(subset=['date', 'timestamp', 'uid']).sum())

Number of same [date/timestamp/uid] combination: 488


In [11]:
# remove the entires with same combinations
df_clean = df_clean.loc[~df_clean.duplicated(subset=['date', 'timestamp', 'uid']),:].copy()

#### C - Removing Abnormal Entries - Same User id With More Than One "isFirst" 

As instruction explained, whenever a user pings to the website for the **first time ever**, it shows in the "isFisrt" column. So no user can have more than one "first time usage," and I am removing these entries.

In [12]:
# check all first time users 
df_first = df_clean[df_clean['isFirst'] == True]

In [13]:
print ("Number of First Time UNIQUE Users:",df_first['uid'].nunique())
print ("Number of Rows With isFirst=True:", len(df_first))

Number of First Time UNIQUE Users: 86598
Number of Rows With isFirst=True: 86643


So basically, there are **45** rows with user ids that appear more than once with "isFirst" as true.
(Some user ids have more than one "isFirst", which is not meaningful, and its probably because of the system malfunctions)

The best way is to keep the rows that appeared first in the dataset and change the others from "isFirst" True to False. However, since it's just 45 rows out of around 86 thousand, and it needs so much time for the system to go over all rows and rewrite them, it would be more logical simply to remove these 45 rows.

In [14]:
# dataframe that has more than one 'isFirst' 
df_first_err = df_first.loc[df_first.duplicated(subset=['uid','isFirst']),:]

In [15]:
# Remove part of dataframe that has more than one "isFirst" occurence from main dataset 

df_cl = pd.concat([df_clean,df_first_err,df_first_err]).drop_duplicates(keep=False)

#### D - Checking the Entries with Delayed "isFirst" 

Also, there may be some cases that could not be successfully recorded their First pings. They may have been occurred because of some system malfunction. I do not have access to the system, but I am checking, so if there are meaningful mistakes, they should be reported to the system administrator. 

In [16]:
# Getting subset of dataset that contain user ideas that have "isFirst" in month Febraury
df_first = df_cl[df_cl['isFirst'] == True]
first_uid = df_first['uid'].unique().tolist()
df_first_all = df_cl[df_cl['uid'].isin(first_uid)]

In [17]:
df_first_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 913546 entries, 26 to 4823565
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   date       913546 non-null  datetime64[ns]
 1   timestamp  913546 non-null  datetime64[ns]
 2   uid        913546 non-null  object        
 3   isFirst    913546 non-null  bool          
 4   utmSource  398995 non-null  object        
dtypes: bool(1), datetime64[ns](2), object(2)
memory usage: 35.7+ MB


913546 entries (pings) have a user ids that had their first ping ever sometime during February 2016.
I will sort the data by **timestamp**, and check for the repeated user ids. For duplicated user ids, the first one (earliest) should have "isFirst" as True. Otherwise, the system could not successfully record the first time ping of some users. 

In [18]:
# Sort the sub dataset that have ids with first time user ids in Feb 
df_first_all_sorted = df_first_all.sort_values(by=['timestamp'])

In [19]:
# Remove the user id duplication and just keep the first one  
df_first_all_sorted = df_first_all_sorted.loc[~df_first_all_sorted.duplicated(subset=['uid']),:].copy()

In [20]:
# Check if there are cases that their first usage did not record correctly
error_first = df_first_all_sorted[df_first_all_sorted['isFirst'] == False]
error_first.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44 entries, 11420 to 4785343
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       44 non-null     datetime64[ns]
 1   timestamp  44 non-null     datetime64[ns]
 2   uid        44 non-null     object        
 3   isFirst    44 non-null     bool          
 4   utmSource  17 non-null     object        
dtypes: bool(1), datetime64[ns](2), object(2)
memory usage: 1.8+ KB


There are 44 cases of system malfucntion for recording the first ping.

In [21]:
error_first_user_list = error_first['uid'].tolist()

# Check the difference between real first occurence and its recorded first

df_first_err = df_first_all[df_first_all['uid'].isin(error_first_user_list)]
df_first_err_grp = df_first_err.groupby(['uid','isFirst']).agg({'date': ['min']})

In [22]:
# can be deleted
display_all(df_first_err_grp)

date
                            min
uid          isFirst           
50569954     False   2016-02-23
             True    2016-02-23
50571665     False   2016-02-23
             True    2016-02-23
50707349     False   2016-02-14
             True    2016-02-14
52010378     False   2016-02-23
             True    2016-02-23
52132056     False   2016-02-01
             True    2016-02-01
...                         ...
76359336     False   2016-02-23
             True    2016-02-23
77784600     False   2016-02-23
             True    2016-02-23
78619047     False   2016-02-02
             True    2016-02-02
79795434     False   2016-02-01
             True    2016-02-01
Xt0mzeFz4emB False   2016-02-24
             True    2016-02-24

[88 rows x 1 columns]

However, it seems that most of the malfunctions record occured on the same day (mostly the second pings are successfuly recorded). So if we want to stick to daily analysis, it would not make much difference and I will keep them in the dataset

## Questions 

### 1 - Daily Active Users Over the Month

In this part, the number of "daily active users" can be seen. I created a new column as **IsRegistered**, since It would be useful to see what portion of the daily users are registered and what portion is not registered. 

In [23]:
def reg_func(usid):
    
    '''This function check whether the user is registered or not, using its id'''
    
    try:
        num = int(usid)
        return("Registered")
    except ValueError: 
        # Handel the exception
        return("Non Registered")

In [24]:
# Create new column for 'registered' and 'non registered users'
df_cl['isRegistered'] = df_cl['uid'].apply(lambda x : reg_func(x))

In [25]:
df_cl.head(5)

,date,timestamp,uid,isFirst,utmSource,isRegistered
0,2016-02-01,2016-02-01 08:00:01.230,53386875,False,Biznesowe+Rewolucje,Registered
1,2016-02-01,2016-02-01 08:00:01.410,70935792,False,None,Registered
2,2016-02-01,2016-02-01 08:00:01.451,IG4TX0lZdTk1,False,None,Non Registered
3,2016-02-01,2016-02-01 08:00:01.556,70128721,False,Biznesowe+Rewolucje,Registered
4,2016-02-01,2016-02-01 08:00:01.673,53386875,False,Biznesowe+Rewolucje,Registered


In [26]:
df_cl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4810055 entries, 0 to 4823565
Data columns (total 6 columns):
 #   Column        Dtype         
---  ------        -----         
 0   date          datetime64[ns]
 1   timestamp     datetime64[ns]
 2   uid           object        
 3   isFirst       bool          
 4   utmSource     object        
 5   isRegistered  object        
dtypes: bool(1), datetime64[ns](2), object(3)
memory usage: 224.8+ MB


In [27]:
# Group by 'date' and 'isRegistered' columns 
df_grp_day = df_cl.groupby(by=['date','isRegistered'], as_index=False).agg({'uid': pd.Series.nunique})

In [28]:
# Stacked bar chart - daily active users

# This column is just added for having better visulization in Altair
df_grp_day['Date'] = df_grp_day['date'].dt.strftime('%b %d')


fig1 = alt.Chart(df_grp_day).transform_joinaggregate(
    tot = 'sum(uid):Q',
    count_true = 'max(uid):Q',  #kalak rashti
    groupby=['Date']
).transform_calculate(
    count_false = "datum.tot - datum.count_true"
).mark_bar(size=10).encode(
    x=alt.X('Date:O', axis = alt.Axis(title = 'Date',labelAngle=90)),
    y=alt.Y('uid:Q', title = 'Daily Active Users'),
    color=alt.Color('isRegistered',legend=alt.Legend(title="")),
    order=alt.Order('isRegistered', sort='ascending'),
    tooltip=[alt.Tooltip('Date', title = 'Date'),
             alt.Tooltip('tot:Q', title = 'Total Users'),
             alt.Tooltip('count_true:Q', title = 'Registered Users'),
             alt.Tooltip('count_false:Q' , title = "Non Registered Users")]
).properties(
    height= 300,
    width = 600 ,title=["Daily Active Users", "February 2016"]
)

chart_a = alt.layer(fig1
    ).configure_title(fontSize=18,font='Helvetica Sans', color='dimgrey', dy=-5
    ).configure_legend(labelFontSize=12, labelColor='k'
    ).configure_axisY(labelPadding=10, titleX=-60, labelFontSize=10,titleFontSize=16, titleColor = 'dimgray', titleFont = 'Helvetica Sans' 
    ).configure_axisX(grid=False,titleY=50,titleFontSize=16, labelFontSize=10, titleColor = 'dimgray', titleFont = 'Helvetica Sans'
    ).configure_axis(gridOpacity = 0.6
    )


chart_a


alt.LayerChart(...)

In [29]:
chart_a.save('chart_a.html')

### 2 -  Daily Retention Curve

The retention Curve is very insightful. It tells us to what extent our product is successful in keeping the users throughout a certain time.

For this specific exercise, the retention curve is calculated for three certain cohorts: Feb 4th, Feb 10th, and Feb 14th.

In [30]:
def cohort_func(data, start_date, end_date="2016-02-29"):
    '''This function takes the dataset and start date outputs the 
    list of cohort users, num of daily users, and retention dataframe for users that have the specfic first date'''
    
    # user ids of cohort 
    cohort_list = data[(data['date'] == start_date)&(data['isFirst']== True)]['uid'].unique().tolist()
    cohort_size = len(cohort_list)
    
    # create a dataframe for cohort users over the month
    df_cohort = data[data['uid'].isin(cohort_list)]
        
    # cohort_data calculates the percentage of unique users who used the app on each day.
    cohort_ret = pd.DataFrame((df_cohort.groupby(['date']).nunique()['uid']/cohort_size))
    cohort_ret.reset_index(inplace=True)
    cohort_ret.columns = ['Date', 'Retention Percentage']
    
    # Add new columns as "Cohort Life Time" and "Cohort Users"
    cohort_ret["Cohort Life Time"] = np.arange(0,len(cohort_ret))
    cohort_ret["Number of Users"] = df_cohort.groupby(['date']).nunique()['uid'].tolist()
    
    return cohort_list, cohort_size, cohort_ret

In [31]:
# Create cohort user list, size, daily retenrion perc, numb of users for 3 cohort dates (4th, 10th, and 14th Feb)

# Cohort 4th Feb
cohort_4 = cohort_func(df_cl, start_date = "2016-02-04")
cohort_list_4 = cohort_4[0]
cohort_size_4 = cohort_4[1]
cohort_ret_4 = cohort_4[2]

# Cohort 10th Feb
cohort_10 = cohort_func(df_cl, start_date = "2016-02-10")
cohort_list_10 = cohort_10[0]
cohort_size_10 = cohort_10[1]
cohort_ret_10 = cohort_10[2]

# Cohort 14th Feb
cohort_14 = cohort_func(df_cl, start_date = "2016-02-14")
cohort_list_14 = cohort_14[0]
cohort_size_14 = cohort_14[1]
cohort_ret_14 = cohort_14[2]

For having a better comparison, I am going to concat all three cohorts' retention dfs. I also added one more column as "Cohort" to show the label of each row. 

In [32]:
# Create all cohorts Size Dfs and Concat them for plotting purpuse

df_cohort_size_4 = pd.DataFrame(data=[[cohort_size_4,'4th Feb']], columns=['Size','Cohort'])
df_cohort_size_10 = pd.DataFrame(data=[[cohort_size_10,'10th Feb']], columns=['Size','Cohort'])
df_cohort_size_14 = pd.DataFrame(data=[[cohort_size_14,'14th Feb']], columns=['Size','Cohort'])

# concat all cohorts size dfs
cohort_size_all = pd.concat([df_cohort_size_4 , df_cohort_size_10, df_cohort_size_14 ], ignore_index=True)

In [33]:
# Concat all cohorts Retention Dfs with adding one column to show Cohort label

# Add new column "Cohort" to all retention dfs 
cohort_ret_4['Cohort']  = "4th Feb"
cohort_ret_10['Cohort'] = "10th Feb"
cohort_ret_14['Cohort'] = "14th Feb"

# Concat all cohorts
cohort_ret_all = pd.concat([cohort_ret_4, cohort_ret_10, cohort_ret_14], ignore_index=True)

In [96]:
# Plot the retention heatmap

## Retention Heat Map
fig2 = alt.Chart(cohort_ret_all).mark_rect().encode(
        alt.X('Cohort Life Time:O', title='Cohort Life - Days'),
        alt.Y('Cohort:O', title= 'Cohort', sort=["4th Feb","10th Feb","14th Feb"]),
        alt.Color('Retention Percentage:Q', title='Retention Percentage', legend = alt.Legend(format=".0%")),
        tooltip=[alt.Tooltip('Date', title = 'Date'),
                 alt.Tooltip('Cohort', title = 'Cohort'),
                 alt.Tooltip('Number of Users:Q', title = 'Number of Users'),
                 alt.Tooltip('Retention Percentage:Q', title = 'Retention Percentage', format=(".2%"))]
).properties(
    height= 200,
    width = 800 ,title=["Daily Cohort Retention Percentage", "February 2016"]
)


chart_b = alt.layer(fig2
    ).configure_title(fontSize=18,font='Helvetica Sans', color='dimgrey', dy=-15
    ).configure_legend(labelFontSize=10, titleColor ='dimgray', titleFont = "Helvetica Sans", labelColor = 'dimgray'
    ).configure_axisY(labelPadding=10, titleX=-60, labelFontSize=10,titleFontSize=13, titleColor = 'dimgray', titleFont = 'Helvetica Sans' 
    ).configure_axisX(grid=False,titleY=30,titleFontSize=13, labelFontSize=10, titleColor = 'dimgray', titleFont = 'Helvetica Sans'
    ).configure_axis(gridOpacity = 1
    )
    

annotation = alt.Chart(cohort_ret_all).mark_text(
    align='center',
    baseline='middle',
    fontSize = 7,
    dx = 0
).encode(
    x='Cohort Life Time:O',
    y=alt.Y('Cohort:O', title= 'Cohort', sort=["4th Feb","10th Feb","14th Feb"]),
    text=alt.Text('Retention Percentage:Q' , format=(".1%"))
)    
    
    
    
chart_b2 = chart_b + annotation
chart_b2



alt.LayerChart(...)

In [35]:
chart_b2.save('chart_b2.html')

In [36]:
# Cohort Total Users - Bar Chart 
fig3 = alt.Chart(cohort_size_all).mark_bar(size=60).encode(
    x=alt.X('Cohort:O', axis = alt.Axis(title = 'Cohort',labelAngle=0), sort=["4th Feb","10th Feb","14th Feb"]),
    y=alt.Y('Size:Q', title = 'Size'),
    color=alt.Color('Cohort:O', legend=None, scale=alt.Scale(domain=['4th Feb', '10th Feb', '14th Feb'],range=['#8c144a', '#4147d0','#476108'])),
    tooltip=[alt.Tooltip('Cohort', title = 'Cohort'),
             alt.Tooltip('Size:Q', title = 'Total Users')]
).properties(
    height= 220,
    width = 400 ,title=["Cohort Total Users", "February 2016"]
)

chart_c = alt.layer(fig3
    ).configure_title(fontSize=14,font='Helvetica Sans', color='dimgrey', dy=-5
    ).configure_legend(labelFontSize=12, titleColor ='dimgray'
    ).configure_axisY(labelPadding=5, titleX=-50, labelFontSize=10,titleFontSize=12, titleColor = 'dimgray', titleFont = 'Helvetica Sans' 
    ).configure_axisX(grid=False,titleY=30,titleFontSize=12, labelFontSize=10, titleColor = 'dimgray', titleFont = 'Helvetica Sans'
    ).configure_axis(gridOpacity = 0.6
    )


chart_c


alt.LayerChart(...)

In [37]:
chart_c.save('chart_c.html')

In [90]:
### Plot Cohort Retention throughout the end of the month


# This column is just added for having better visulization in Altair
cohort_ret_all['date'] =  cohort_ret_all['Date'].dt.strftime('%b %d')


# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['date'], empty='none')


fig3_b = alt.Chart(cohort_ret_all).mark_line().encode(
    x=alt.X('date:O', axis = alt.Axis(title = 'Date',labelAngle=90)),
    y=alt.Y('Retention Percentage:Q', axis = alt.Axis(title = 'Retention Percentage',format="%"),scale=alt.Scale(domain=(0,1.02))),
    color = alt.Color("Cohort:O",legend=alt.Legend(title="Cohort"), scale=alt.Scale(domain=['4th Feb', '10th Feb', '14th Feb'],range=['#8c144a', '#4147d0','#476108'])),
    tooltip = [alt.Tooltip('date', title = 'Date'),
               alt.Tooltip('Retention Percentage:Q', title = 'Retention Percentage',format=(".2%")),
               alt.Tooltip('Cohort:O', title='cohort')]
).properties(
    height= 400,
    width = 800 ,title=["Retention Percentage", "Across Cohorts"]
)

                          

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(cohort_ret_all).mark_point().encode(
    x='date:O',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = fig3_b.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(0.5), alt.value(0))
)


# Draw text labels near the points, and highlight based on selection
text = fig3_b.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'Retention Percentage:Q', alt.value(' '), format=(".2%"))
)

# Draw a rule at the location of the selection
rules = alt.Chart(cohort_ret_all).mark_rule(color='gray').encode(
    x='date:O',
).transform_filter(
    nearest
)

chart_c2 = alt.layer(fig3_b, selectors, points, rules, text
    ).configure_title(fontSize=18,font='Helvetica Sans', color='dimgrey', dy=-5
    ).configure_legend(labelFontSize=12, title=None
    ).configure_axisY(labelPadding=10, titleX=-60, labelFontSize=10,titleFontSize=16, titleColor = 'dimgray', titleFont = 'Helvetica Sans' 
    ).configure_axisX(grid=False,titleY=50,titleFontSize=16, labelFontSize=10, titleColor = 'dimgray', titleFont = 'Helvetica Sans'
    ).configure_axis(gridOpacity = 0.6
    )


chart_c2


alt.LayerChart(...)

In [91]:
chart_c2.save('chart_c2.html')

### 3 -  Source & User Analysis

In this part, several metrics are defined to measure the value of sources and users for the business. Identifying the best and worst users can help us improve points with more potential

### 3-1 Source Analysis

In this part, I am going to look closer to the traffic sources, 'utmSource'.
Almost one third of the traffic sources, **1667897** is "Not Available".

In [40]:
print ("Number of NaN entries in utmSource Column:",df_cl['utmSource'].isnull().sum())
print ("Number of Unique Traffic Sources:", df_cl['utmSource'].nunique())
df_cl = df_cl.replace(np.nan, 'NA', regex=True)

Number of NaN entries in utmSource Column: 1667896
Number of Unique Traffic Sources: 52


In [41]:
source_list = df_cl['utmSource'].unique()
print ("Traffic Sources:\n\n",source_list)

Traffic Sources:

 ['Biznesowe+Rewolucje' 'NA' 'answers' 'program' 'MosaLingua+FR'
 'Just-In-Time+Travels+Newsletter' 'shmoop_left' 'twitter' 'handbook'
 'salesmanago' 'youtube' 'Grub+Street' 'contenthub' 'blog'
 'MembershipCancellation' 'facebook' 'dict' 'other' 'tapjoy' 'liveintent'
 'placement' 'Facebook_org' 'Blog_org' 'Wise+Ink+Master+Email+List'
 "Sarah+Doody's+UX+Notebook" 'google' 'TrialIntro' 'LinkedIn_org' 'Bing'
 'pandora' 'nettedbythewebbys' 'taboola' 'Twitter_org' 'GPlus_org' 'brand'
 're:+charity' 'outbrain' 'email-sendgrid' 'facebook.com' 'Pre-quote+List'
 'SummerInvite' 'digg' 'Sticky Blogging Secrets' 'Book+Quote'
 'blogger_outreach' 'display' 'sendgrid.com' 'shmoop_logo' 'cafemom.com'
 'shmoop_right' 'bing' 'card' 'gsp']


Some sources are recorded in slightly different names, for instance, 'Twitter_org' and 'twitter.' 
First, some modifications need to be made. 

In [42]:
def src_correction (word): 
    '''This function modifies the names of traffic sources'''
    
    word = word.lower()
    word = re.sub('blog.*','blog', word)
    word = re.sub('biznesowe\+rewolucje','biznesowe rewolucje', word)
    word = re.sub('blogger_outreach','blogger outreach', word)
    word = re.sub('book\+quote','book quote', word)
    word = re.sub('cafemom.*','cefemom', word)
    word = re.sub('email-sendgrid','sendgrid', word)
    word = re.sub('facebook.*','facebook', word)
    word = re.sub('gplus_org','google', word)  #gplus_org is considered as google plus and added to google
    word = re.sub('grub.*','grub street', word)
    word = re.sub('just.*','just in time travels newsletter', word)
    word = re.sub('linkedin.*','linkedin', word)
    word = re.sub('mosalingua\+fr','mosalingua fr', word)
    word = re.sub('pre-quote\+list','pre quote list', word)
    word = re.sub('re:\+charity','charity', word)
    word = re.sub("sarah\+doody's\+ux\+notebook","sarah doody notebook", word)
    word = re.sub('sendgrid.*','sendgrid', word)
    word = re.sub('shmoop.*','shmoop', word)
    word = re.sub('twitter.*','twitter', word)
    word = re.sub('wise\+ink\+master\+email\+list','wise ink master email list', word)
    
    return word

In [43]:
# Apply source name correction function on 'utmSource' column
df_cl['utmSource'] = df_cl['utmSource'].apply(lambda x : src_correction(x))

For source analysis, based on the main goals of a business, some metrics should be defined. 
Here, I am defining some metrics based on my perception of the nature of the business, although it would be better to come up with metrics in collaboration with other teams.

### 3 - 1 - 1 Daily Unique Actice Users per Source

One metric that I believe could be valuable is: **Daily Unique Active Users per Source**
This metric is valuable because it shows the power of each source as the gate for our product/website. 

Be careful! Some users may use the product from various sources each day. Therefore, some users may be counted several times: 

In [44]:
# Number of 'uniques sources' for each user in each day
df_cl_grp_user_day = df_cl.groupby(['date','uid'])['utmSource'].nunique().reset_index()

# Number of cases that users used more than 1 source in some days. 
num_multi_src = len(df_cl_grp_user_day[df_cl_grp_user_day['utmSource'] > 1])
all_cases = len(df_cl_grp_user_day)
print ("Number of cases that a user used more than 1 traffic source in some days:", num_multi_src)
print ("Percentage of cases that a user used more than 1 traffic source in some days:", round((num_multi_src*100/all_cases),2))

Number of cases that a user used more than 1 traffic source in some days: 1002
Percentage of cases that a user used more than 1 traffic source in some days: 0.03


1002 out of around 3.3 million cases, users used more than one source in some days. This is a very low number so basically, we can drop these rows, and it would not impact the final result very much.

In [45]:
# Users with More than 1 sources
multi_srcs = df_cl_grp_user_day[df_cl_grp_user_day['utmSource'] > 1]
multi_srcs_date_uid = multi_srcs[['date','uid']]

In [46]:
# remove the entries that have ids with more than 1 source 
df_cl_src_uniq = pd.merge(df_cl, multi_srcs_date_uid, on=['date','uid'], how='outer', indicator=True).query("_merge != 'both'").drop('_merge', axis=1).reset_index(drop=True)

Now we have the dataframe that shows each user used one specific source to use our product. So, we need to count the daily unique users for each traffic source. 

In [47]:
# Clean DataFrame Info
df_cl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4810055 entries, 0 to 4823565
Data columns (total 6 columns):
 #   Column        Dtype         
---  ------        -----         
 0   date          datetime64[ns]
 1   timestamp     datetime64[ns]
 2   uid           object        
 3   isFirst       bool          
 4   utmSource     object        
 5   isRegistered  object        
dtypes: bool(1), datetime64[ns](2), object(3)
memory usage: 224.8+ MB


In [48]:
# Count Unique Users per Source Per Day
df_uniqusers_perday_persrc = df_cl_src_uniq.groupby(["date","utmSource"]).nunique()['uid'].reset_index()

In [49]:
df_uniqusers_perday_persrc.head()

,date,utmSource,uid
0,2016-02-01,answers,5703
1,2016-02-01,bing,1
2,2016-02-01,biznesowe rewolucje,43938
3,2016-02-01,blog,35
4,2016-02-01,book quote,2


Some sources have a minimal number of pings, so let's focus more on the top ones. 
The average daily users for **all sources** is around 116098. In other words, on average, each day, 116098 unique users from various sources use the product. So I assign the threshold as **580 (~0.5%)** of the daily users to filter out some sources that do not bring much value into the business (although these sources can be the topic of the other research and see how we can boost their users). 

In [50]:
# Average Daily Users for All Sources 
df_uniqusers_perday_persrc.groupby('date')['uid'].sum().mean()

116098.20689655172

In [51]:
# filter out the major traffic sources based on their daily traffic 
df_uniqusers_perday_persrc_top = df_uniqusers_perday_persrc[df_uniqusers_perday_persrc['uid'] > 580]
print ("Number of Sources that Have at least 580 Unique Users:", df_uniqusers_perday_persrc_top['utmSource'].nunique() )

Number of Sources that Have at least 580 Unique Users: 11


In [52]:
top_src = df_uniqusers_perday_persrc_top['utmSource'].unique().tolist()

In [53]:
# Plot the Number of daily active users for each source through the month (Feb 2016)

# This column is just added for having better visulization in Altair
df_uniqusers_perday_persrc_top['Date'] =  df_uniqusers_perday_persrc_top['date'].dt.strftime('%b %d')

# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['Date'], empty='none')


fig4 = alt.Chart(df_uniqusers_perday_persrc_top).mark_line().encode(
    x=alt.X('Date:O', axis = alt.Axis(title = 'Date',labelAngle=90)),
    y=alt.Y('uid:Q', axis = alt.Axis(title = 'Number of Unique Users')),
    color = alt.Color("utmSource:O",legend=alt.Legend(title="Source"), scale=alt.Scale(range=["#d5c56f","#e78c19","#fa8775","#ea5f94","#34cda5","#9d02d7","#0000ff","#6b9c2e","#c95d1a","#7e745b","#91a9e7"])),
    tooltip = [alt.Tooltip('Date', title = 'Date'),
               alt.Tooltip('uid:Q', title = 'Number of Users'),
               alt.Tooltip('utmSource:O', title='Traffic Source')]
).properties(
    height= 400,
    width = 800 ,title=["Daily Unique Users", "Across Traffic Sources"]
)




# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(df_uniqusers_perday_persrc_top).mark_point().encode(
    x='Date:O',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = fig4.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(0.5), alt.value(0))
)


# Draw text labels near the points, and highlight based on selection
text = fig4.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'uid:Q', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart(df_uniqusers_perday_persrc_top).mark_rule(color='gray').encode(
    x='Date:O',
).transform_filter(
    nearest
)


                          
chart_d = alt.layer(fig4, selectors, points, rules, text
    ).configure_title(fontSize=18,font='Helvetica Sans', color='dimgrey', dy=-5
    ).configure_legend(labelFontSize=12, title=None
    ).configure_axisY(labelPadding=10, titleX=-60, labelFontSize=10,titleFontSize=16, titleColor = 'dimgray', titleFont = 'Helvetica Sans' 
    ).configure_axisX(grid=False,titleY=50,titleFontSize=16, labelFontSize=10, titleColor = 'dimgray', titleFont = 'Helvetica Sans'
    ).configure_axis(gridOpacity = 0.6
    )


chart_d


alt.LayerChart(...)

In [54]:
chart_d.save('chart_d.html')

### 3 - 1 - 2 Daily User Activity Distribution Per Source

This metric shows how consistent users' activities are across various sources throughout the month. In other words, we want to investigate the **user activity distribution of each source**. 

In [55]:
# df for number of active days for each user and source

df_uniqday_peruser_persrc = df_cl.groupby(['uid','utmSource']).nunique()['date'].reset_index()
df_uniqday_peruser_persrc.head()

,uid,utmSource,date
0,007lqy8pw8PF,na,1
1,008wXBXcFOF5,biznesowe rewolucje,1
2,00BBLeYvGxwU,na,1
3,00Dd47PrCevO,na,1
4,00FhDR8eh9W0,na,2


In [56]:
# df for number of total unique users for each source and number of active days 
df_uniqday_alluser_persrc = df_uniqday_peruser_persrc.groupby(['date','utmSource']).agg({'uid':['count']}).reset_index()

df_uniqday_alluser_persrc.columns = ['Number of Days','utmSource','Daily Users of Source']

In [57]:
# Calculate percentage of users activity from all month activity for each source

# Total users of each source 
df_uniqday_alluser_persrc['Total Users'] = df_uniqday_alluser_persrc.groupby('utmSource')['Daily Users of Source'].transform('sum')

# Percentage of daily activity 
df_uniqday_alluser_persrc['Perc of User Activity'] = df_uniqday_alluser_persrc['Daily Users of Source']/df_uniqday_alluser_persrc['Total Users']

In [58]:
df_uniqday_alluser_persrc.head()

,Number of Days,utmSource,Daily Users of Source,Total Users,Perc of User Activity
0,1,answers,1161,14979,0.077509
1,1,biznesowe rewolucje,5173,86287,0.059951
2,1,blog,3,46,0.065217
3,1,contenthub,11,182,0.060440
4,1,dict,31,342,0.090643


In [59]:
top_src.remove('na')

In [60]:
df_uniqday_alluser_persrc_top = df_uniqday_alluser_persrc[df_uniqday_alluser_persrc['utmSource'].isin(top_src)]

In [61]:
# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['Date'], empty='none')


fig5 = alt.Chart(df_uniqday_alluser_persrc_top).mark_line().encode(
    x=alt.X('Number of Days:O', axis = alt.Axis(title = 'Number of Active Days',labelAngle=90)),
    y=alt.Y('Perc of User Activity:Q', axis = alt.Axis(title = 'Activity Perc'),scale=alt.Scale(domain=(.005,0.12))),
    color = alt.Color("utmSource:O",legend=alt.Legend(title="Source"), scale=alt.Scale(range=["#d5c56f","#e78c19","#fa8775","#ea5f94","#34cda5","#9d02d7","#0000ff","#6b9c2e","#c95d1a","#7e745b","#91a9e7"])),
    tooltip = [alt.Tooltip('utmSource:O', title='Traffic Source'),
               alt.Tooltip('Number of Days', title = 'Number of Active Days'),
               alt.Tooltip('Perc of User Activity:Q', title = 'User Activity(%)',format=(".2%")),
               alt.Tooltip('Daily Users of Source:Q', title = 'Number of Users')
               ]
).properties(
    height= 400,
    width = 800 ,title=["Users Activty Distribution", "Across Traffic Sources"]
)





             
chart_e = alt.layer(fig5
    ).configure_title(fontSize=18,font='Helvetica Sans', color='dimgrey', dy=-5
    ).configure_legend(labelFontSize=12, title=None
    ).configure_axisY(labelPadding=10, titleX=-60, labelFontSize=10,titleFontSize=16, titleColor = 'dimgray', titleFont = 'Helvetica Sans' 
    ).configure_axisX(grid=False,titleY=50,titleFontSize=16, labelFontSize=10, titleColor = 'dimgray', titleFont = 'Helvetica Sans'
    ).configure_axis(gridOpacity = 0.6
    )


chart_e


alt.LayerChart(...)

In [62]:
chart_e.save('chart_e.html')

### 3 - 2 Users Analysis

In [63]:
# Some User Main Data
print ("Number of unique users in February 2016:", df_cl['uid'].nunique())
print ("Number of unique Registered users in Feb 2016:", df_cl[df_cl['isRegistered'] == "Registered"]['uid'].nunique())
print ("Number of unique NoN Registered users in Feb 2016:", df_cl[df_cl['isRegistered'] == "Non Registered"]['uid'].nunique())

Number of unique users in February 2016: 246840
Number of unique Registered users in Feb 2016: 180009
Number of unique NoN Registered users in Feb 2016: 66831


In order to analysis the best and worst users, certain metrics should be defined. Again thiese metrics may vary business to business and based on the major goals of the buinses. For the sake of the project, I am defining metrics based on my insight and perception.  

In [64]:
# Top Users

df_cl_top_user = df_cl.groupby('uid').count().sort_values(by='timestamp',ascending=False)

In [65]:
df_cl_top_user.describe(percentiles = [.05,.1,.25,.5,.75,.9,.99])

,date,timestamp,isFirst,utmSource,isRegistered
count,246840.000000,246840.000000,246840.000000,246840.000000,246840.000000
mean,19.486530,19.486530,19.486530,19.486530,19.486530
std,32.045012,32.045012,32.045012,32.045012,32.045012
min,1.000000,1.000000,1.000000,1.000000,1.000000
5%,2.000000,2.000000,2.000000,2.000000,2.000000
10%,3.000000,3.000000,3.000000,3.000000,3.000000
25%,7.000000,7.000000,7.000000,7.000000,7.000000
50%,17.000000,17.000000,17.000000,17.000000,17.000000
75%,26.000000,26.000000,26.000000,26.000000,26.000000
90%,33.000000,33.000000,33.000000,33.000000,33.000000


### 3 - 2 - 1  Average Daily Ping 

"Average Daily Ping" can be a good metric. This shows to what extent each user is involved to our business. 

In order to calculate this metric, I need to group the df per day and user id and count the occurence (timestamp can be used to count the occuerenc). Then the output df should be grouped per user id and calculate the mean based on the total days.  

##### Top Users -> Including NA Source

In [66]:
# Grouping the df into [id,date] and count the occurence

df_cl_grp_user_day = df_cl.groupby(['uid','date'])['timestamp'].count().reset_index()

In [67]:
# Group the output df per user id and calculate the average 

df_avgdaily_ping = df_cl_grp_user_day.groupby(['uid']).mean().reset_index()

In [68]:
df_avgdaily_ping.describe(percentiles = [.05,.1,.25,.5,.75,.9,.995])

,timestamp
count,246840.000000
mean,1.716204
std,3.683295
min,1.000000
5%,1.000000
10%,1.000000
25%,1.000000
50%,1.200000
75%,1.750000
90%,2.500000


So let's dig more into **top 1000 (~0.4%)*** users.  

In [69]:
# Seperate top 1000 users based on average daily ping

df_avgdaily_ping_top1000 = df_avgdaily_ping.sort_values(by='timestamp',ascending=False).iloc[:1000]

# List of top 1000 users
top_users = df_avgdaily_ping_top1000['uid'].tolist()

# DF of top 1000 users 
df_top_users = df_cl[df_cl['uid'].isin(top_users)]

In [70]:
df_top_users.describe()

,date,timestamp,uid,isFirst,utmSource,isRegistered
count,245646,245646,245646,245646,245646,245646
unique,29,245637,1000,2,14,2
top,2016-02-16 00:00:00,2016-02-27 02:03:22.817000,vwLJO2uDWrXr,False,na,Non Registered
freq,10326,2,2807,244939,159882,232812
first,2016-02-01 00:00:00,2016-02-01 08:00:14.860000,NaN,NaN,NaN,NaN
last,2016-02-29 00:00:00,2016-03-01 07:59:55.129000,NaN,NaN,NaN,NaN


In [71]:
df_top_users_grp_usr = df_top_users.groupby(['uid'])['utmSource'].nunique().reset_index()

In [72]:
mlt_src_top_users = df_top_users_grp_usr[df_top_users_grp_usr['utmSource'] > 1]['uid'].tolist()
print ("The number of multi sources of top 1000 users:", len(mlt_src_top_users))

The number of multi sources of top 1000 users: 2


There are just **2 (0.2%)** top users that have more than one sources. So we can simply remove them.  

In [73]:
# Remove 2 users that have more than one source

df_top_users = df_top_users[~df_top_users['uid'].isin(mlt_src_top_users)] 

In [74]:
df_top_users_unq = df_top_users.loc[~df_top_users.duplicated(subset=['uid']),:]
df_top_users_unq.head()

,date,timestamp,uid,isFirst,utmSource,isRegistered
28,2016-02-01,2016-02-01 08:00:14.860,9Lfb2Zkyi8Ps,False,na,Non Registered
41,2016-02-01,2016-02-01 08:00:22.370,eGK6mbHzvwRD,False,na,Non Registered
75,2016-02-01,2016-02-01 08:00:49.726,aQPgQRgPH6b8,False,na,Non Registered
312,2016-02-01,2016-02-01 08:03:46.679,Mtq6cVxORH7P,False,na,Non Registered
390,2016-02-01,2016-02-01 08:04:40.326,saJJ9pkcq4Rb,False,contenthub,Non Registered


In [75]:
# Sources of Top 1000 Users - Bar Chart 
fig6 = alt.Chart(df_top_users_unq).mark_bar(size=10).encode(
    x=alt.X('utmSource:O', axis = alt.Axis(title = 'Traffic Source',labelAngle=90)),
    y=alt.Y('count(utmSource):Q', axis = alt.Axis(title = 'Number of Top Users')),
    color = alt.Color("utmSource:O",legend=alt.Legend(title="Source"), scale=alt.Scale(range=["#d5c56f","#e78c19","#fa8775","#ea5f94","#34cda5","#9d02d7","#0000ff","#6b9c2e","#c95d1a","#7e745b","#91a9e7","#929482","#e81515","#27a18e"])),
    tooltip=[alt.Tooltip('utmSource', title = 'Source'),
             alt.Tooltip('count(utmSource):Q', title = 'Total Users')]
).properties(
    height= 420,
    width = 400 ,title=["Sources of Top 1000 Users", "February 2016"]
)

chart_f = alt.layer(fig6
    ).configure_title(fontSize=14,font='Helvetica Sans', color='dimgrey', dy=-5
    ).configure_legend(labelFontSize=12, titleColor ='dimgray'
    ).configure_axisY(labelPadding=5, titleX=-50, labelFontSize=10,titleFontSize=12, titleColor = 'dimgray', titleFont = 'Helvetica Sans' 
    ).configure_axisX(grid=False,titleY=80,titleFontSize=12, labelFontSize=10, titleColor = 'dimgray', titleFont = 'Helvetica Sans'
    ).configure_axis(gridOpacity = 0.6
    )

chart_f

alt.LayerChart(...)

In [76]:
chart_f.save('chart_f.html')

Most of the top users **83%** their sources are not identified. So, let's repeat the same steps excluding the entries with "NA" source. 

##### Top Users ->  Excluding NA Source

In [77]:
df_cl_wo_na = df_cl[df_cl['utmSource'] != 'na']

In [78]:
# Grouping the df into [id,date] and count the occurence

df_cl_wo_na_grp_user_day = df_cl_wo_na.groupby(['uid','date'])['timestamp'].count().reset_index()

In [79]:
# Group the output df per user id and calculate the average 

df_avgdaily_ping_wo_na = df_cl_wo_na_grp_user_day.groupby(['uid']).mean().reset_index()

In [80]:
df_avgdaily_ping_wo_na.describe(percentiles = [.05,.1,.25,.5,.75,.9,.998])

,timestamp
count,147737.000000
mean,1.469193
std,1.457093
min,1.000000
5%,1.000000
10%,1.000000
25%,1.000000
50%,1.166667
75%,1.533333
90%,2.153846


so let's dig into **top 500(~0.2%)** users (based on daily average pings) 

In [81]:
# Seperate top 500 users based on average daily ping

df_avgdaily_ping_top500 = df_avgdaily_ping_wo_na.sort_values(by='timestamp',ascending=False).iloc[:500]

# List of top 500 users
top_users_wo_na = df_avgdaily_ping_top500['uid'].tolist()

# DF of top 500 users 
df_top_users_wo_na = df_cl_wo_na[df_cl_wo_na['uid'].isin(top_users_wo_na)]

In [82]:
df_top_users_wo_na.describe()

,date,timestamp,uid,isFirst,utmSource,isRegistered
count,129865,129865,129865,129865,129865,129865
unique,29,128831,500,2,17,2
top,2016-02-12 00:00:00,2016-02-25 19:01:10.168000,vwLJO2uDWrXr,False,biznesowe rewolucje,Non Registered
freq,5538,17,2807,129816,58958,101882
first,2016-02-01 00:00:00,2016-02-01 08:00:06.284000,NaN,NaN,NaN,NaN
last,2016-02-29 00:00:00,2016-03-01 07:59:00.142000,NaN,NaN,NaN,NaN


In [83]:
df_top_users_grp_usr_wo_na = df_top_users_wo_na.groupby(['uid'])['utmSource'].nunique().reset_index()

In [84]:
mlt_src_top_users_wo_na = df_top_users_grp_usr_wo_na[df_top_users_grp_usr_wo_na['utmSource'] > 1]['uid'].tolist()
print ("The number of multi sources of top 1000 users:", len(mlt_src_top_users_wo_na))

The number of multi sources of top 1000 users: 21


There are just **21 (4%)** top users that have more than one sources. So we can simply remove them. 

In [85]:
# Remove 21 users that have more than one source

df_top_users_wo_na = df_top_users_wo_na[~df_top_users_wo_na['uid'].isin(mlt_src_top_users_wo_na)] 

In [86]:
df_top_users_unq_wo_na = df_top_users_wo_na.loc[~df_top_users_wo_na.duplicated(subset=['uid']),:]
df_top_users_unq_wo_na.head()

,date,timestamp,uid,isFirst,utmSource,isRegistered
12,2016-02-01,2016-02-01 08:00:06.284,F7RPRxLBMKN9,False,biznesowe rewolucje,Non Registered
390,2016-02-01,2016-02-01 08:04:40.326,saJJ9pkcq4Rb,False,contenthub,Non Registered
434,2016-02-01,2016-02-01 08:05:05.509,57123875,False,biznesowe rewolucje,Registered
810,2016-02-01,2016-02-01 08:09:37.042,rN6LJTYjmQdb,False,biznesowe rewolucje,Non Registered
1088,2016-02-01,2016-02-01 08:12:41.471,EJihOVKqyQXv,False,other,Non Registered


In [87]:
# Sources of Top 500 Users(Without NA Source) - Bar Chart 
fig7 = alt.Chart(df_top_users_unq_wo_na).mark_bar(size=10).encode(
    x=alt.X('utmSource:O', axis = alt.Axis(title = 'Traffic Source',labelAngle=90)),
    y=alt.Y('count(utmSource):Q', axis = alt.Axis(title = 'Number of Top Users')),
    color = alt.Color("utmSource:O",legend=alt.Legend(title="Source"), scale=alt.Scale(range=["#d5c56f","#e78c19","#fa8775","#ea5f94","#34cda5","#9d02d7","#0000ff","#6b9c2e","#c95d1a","#7e745b","#91a9e7","#929482","#e81515","#27a18e","#f3a384","#1bf542","#c7a3cd"])),
    tooltip=[alt.Tooltip('utmSource', title = 'Source'),
             alt.Tooltip('count(utmSource):Q', title = 'Total Users')]
).properties(
    height= 420,
    width = 400 ,title=["Sources of Top 500 Users", "February 2016"]
)

chart_g = alt.layer(fig7
    ).configure_title(fontSize=14,font='Helvetica Sans', color='dimgrey', dy=-5
    ).configure_legend(labelFontSize=12, titleColor ='dimgray'
    ).configure_axisY(labelPadding=5, titleX=-50, labelFontSize=10,titleFontSize=12, titleColor = 'dimgray', titleFont = 'Helvetica Sans' 
    ).configure_axisX(grid=False,titleY=80,titleFontSize=12, labelFontSize=10, titleColor = 'dimgray', titleFont = 'Helvetica Sans'
    ).configure_axis(gridOpacity = 0.6
    )


chart_g


alt.LayerChart(...)

In [88]:
chart_g.save('chart_g.html')